# Units and Particles

This notebook introduces some of the most commonly used functionality when working with PlasmaPy...

## Units

In scientific programming, it is common to represent physical quantities as numbers.

In [ ]:
distance_in_miles = 5
time_in_hours = 60
velocity_in_mph = distance_in_miles / time_in_hours

Representing physical quantities as numbers comes with some risks, such as accidentally performing operations with different units like `time_in_seconds + time_in_hours` or incompatible units like `length + time`.

To avoid situations like this, we can use `astropy.units`. It's standard to import this subpackage as `u`.

In [ ]:
import astropy.units as u

We can combine a number with a unit by multiplying them together.

In [ ]:
distance = 8 * u.km

The `distance` object that we created is a `Quantity` that combines a number (or array of numbers) with a physical unit.

In [ ]:
type(distance)

Operations between `Quantity` objects handle unit operations automatically.

In [ ]:
time = 60 * u.hr
velocity = distance / time
print(velocity)

We can convert a `Quantity` to different units using the `.to` method of a `Quantity`. This method accepts strings that represent a unit (including compound units) or a unit object.

In [ ]:
velocity.to("m/s")

In [ ]:
velocity.to(u.m / u.s)

The `.si` and `.cgs` attributes provide the `Quantity` in SI or CGS units, respectively.

In [ ]:
velocity.si

In [ ]:
velocity.cgs

We can add two `Quantity` objects together if they have different units, as long as they have compatible physical types.

In [ ]:
1 * u.m + 1 * u.cm

When we attempt operations with physically incompatible units, an exception is raised. This helps us prevent common errors.

In [ ]:
3 * u.m + 3 * u.s

We can create dimensionless `Quantity` objects too.

In [ ]:
3 * u.dimensionless_unscaled

We can use `astropy.units` to help create recipes with ridiculous units.

In [ ]:
volume = u.barn * u.Mpc
volume.to(u.imperial.tsp)

It is common for plasma physicists to use electron-volts (eV) as units of temperature, or rather as the temperature divided by the Boltzmann constant. When working with a units package, this can cause some problems.  To handle this, `astropy.units` has built-in *equivalencies*. 

In [ ]:
thermal_energy_per_particle = 1 * u.eV
thermal_energy_per_particle.to("K", equivalencies=u.temperature_energy())

We can access most commonly needed physical constants from `astropy.constants`.

In [ ]:
from astropy import constants

In [ ]:
constants.c

In [ ]:
constants.k_B

## Working with PlasmaPy particles

The `plasmapy.particles` subpackage is...

In [ ]:
from plasmapy.particles import *

### Functions

There are a few functions that provide us with information about the different particles that show up in plasmas. The input of these functions is a representation of a particle, such as a string for the atomic symbol or the element name.

In [ ]:
atomic_number("Fe")

In [ ]:
atomic_symbol("oganesson")

We can provide a number to represent the atomic number.

In [ ]:
element_name(26)

We can also use standard symbols or the names of particles.

In [ ]:
electric_charge("p+")

In [ ]:
charge_number("electron")

We can even use the symbols for many particles directly.  In a Jupyter notebook, type `\alpha` and press tab to create "α".

In [ ]:
particle_mass("α")

There is flexibility in how we represent particles, including ions.  

In [ ]:
particle_mass("Fe-56 13+")

In [ ]:
particle_mass("iron-56 +13")

In [ ]:
particle_mass("iron-56+++++++++++++")

Most of these functions take additional arguments, with `Z` typically representing the charge number of an ion and `mass_numb` representing the mass number of an isotope. These arguments are *keyword-only* to avoid ambiguity.

In [ ]:
particle_mass("Fe", Z=13, mass_numb=56)

### Classes

Up until now, we have been using functions that accept representations of particles and then return particle properties. With the `Particle` class, we can create particle *objects*.

In [ ]:
proton = Particle("p+")
electron = Particle("electron")
iron56_nuclide = Particle("Fe", Z=26, mass_numb=56)

After doing that, we can access the properties of these particles as attributes.

In [ ]:
proton.mass

In [ ]:
electron.charge

In [ ]:
electron.charge_number

In [ ]:
iron56_nuclide.binding_energy

We can get antiparticles too.  There's the `antiparticle` attribute, and we can also use a tilde as an invert operator.

In [ ]:
electron.antiparticle

In [ ]:
~proton

Sometimes we want to use a particle with custom properties.  For that we can use the `CustomParticle` class.

In [ ]:
custom_particle = CustomParticle(
    9.27e-26 * u.kg, 13.6 * constants.e.si, symbol="Fe 13.6+"
)

In [ ]:
custom_particle.mass

In [ ]:
custom_particle.charge

In [ ]:
custom_particle.symbol

If we do not include one of the physical quantities, it gets set to `numpy.nan` in the appropriate units.

In [ ]:
CustomParticle(9.27e-26 * u.kg).charge

When we add `Particle` and/or `CustomParticle` objects together, we get a `ParticleList` that includes all the particles.

In [ ]:
proton + electron + custom_particle

We can use a `ParticleList` to access the properties of multiple particles at once.

In [ ]:
iron_ions = ParticleList(["Fe 12+", "Fe 13+", "Fe 14+"])

In [ ]:
iron_ions.mass

In [ ]:
iron_ions.charge

In [ ]:
iron_ions.symbols

### Nuclear reactions

We can use `plasmapy.particles` to calculate the energy of a nuclear reaction.  To do this, we redefined the `>` operator.

In [ ]:
deuteron = Particle("D+")
triton = Particle("T+")
alpha = Particle("α")
neutron = Particle("n")

In [ ]:
energy = deuteron + triton > alpha + neutron

In [ ]:
energy.to("MeV")